In [3]:
import os
import json
import psycopg2
from glob import glob
from importlib.resources import files
from config.db_config import DBConfig
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://192.168.1.20:1318/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_prompt(title:str,
               contents:str):
    return f"""당신은 뉴스/기사 분석 전문가입니다.

본문에는 뉴스/기사 내용과 관련없는 단어가 많습니다.
뉴스/기사는 해운과 관련된 내용입니다.

[제목]
{title}

아래 [본문] 내용에서 뉴스 내용만 정확히 추출해서 설명해주세요.
답변은 무조건 추출한 뉴스 내용만 생성합니다.

[본문]
{contents}
"""

def LLM_Call(prompt:str):
    chat_response = client.chat.completions.create(
        model="Qwen/Qwen2-VL-72B-Instruct",
        messages=[
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": f"{prompt}"},
        ],
        temperature=0.001,
        top_p=0.001,
        max_tokens=4096,
        extra_body={
            "repetition_penalty": 1.03,
        },
    )
    return chat_response.choices[0].message.content

In [4]:
# from tqdm import tqdm
# for sub_path in ["drybulk_cargo"]:
#     pathes = str(files("data").joinpath(f"raw/{sub_path}/*.json"))
#     pathes = glob(pathes)
#     for path in pathes:
#         if "hellenicshippingnews" not in path:
#             continue
#         filename = path.split('/')[-1]
#         print(path)
#         with open(path, "r") as f:
#             dataset = json.load(f)
#         SOURCES = []
#         for data in tqdm(dataset):
#             CONTENTS = data['content']
#             TITLE = data['title']
#             prompt = get_prompt(TITLE, CONTENTS)
#             response = LLM_Call(prompt)
#             data['content'] = response
#             SOURCES.append(data)
#             if len(SOURCES) % 100 == 0:
#                 with open(f"{filename}", "w", encoding="utf-8") as f:
#                     json.dump(SOURCES, f, ensure_ascii=False, indent=4)
# with open(f"{filename}", "w", encoding="utf-8") as f:
#     json.dump(SOURCES, f, ensure_ascii=False, indent=4)

/workspace/kobc-agent-service/module/report_agent/retrieval_module/data/raw/drybulk_cargo/hellenicshippingnews.json


  0%|          | 0/900 [00:00<?, ?it/s]

100%|██████████| 900/900 [1:32:15<00:00,  6.15s/it]  


In [7]:
import os
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

load_dotenv()

LLM = ChatHuggingFace(
    llm=HuggingFaceEndpoint(
    endpoint_url=os.getenv("ENDPOINT_URL"),
    huggingfacehub_api_token=os.getenv("HF_API_TOKEN"),
    max_new_tokens=4096,
    top_k=1,
    top_p=0.95,
    temperature=0.001,
    repetition_penalty=1.03,
    # stop_sequences=[os.getenv("END_SUFFIX"),os.getenv("SUFFIX")]
    # stop_sequences=[os.getenv("SUFFIX"),os.getenv("END_SUFFIX")]
    # stop_sequences=[os.getenv("SUFFIX")]
    stop_sequences=[os.getenv("END_SUFFIX")]
    ),
    model_id=os.getenv("MODEL_ID")
)

# 400,000

In [8]:
from tqdm import tqdm
for sub_path in ["drybulk_cargo"]:
    pathes = str(files("data").joinpath(f"raw/{sub_path}/*.json"))
    pathes = glob(pathes)
    for path in pathes:
        if "shippingnewsnet" not in path:
            continue
        filename = path.split('/')[-1]
        print(path)
        with open(path, "r") as f:
            dataset = json.load(f)
        SOURCES = []
        for data in tqdm(dataset):
            CONTENTS = data['content']
            TITLE = data['title']
            prompt = get_prompt(TITLE, CONTENTS)
            response = LLM.invoke(prompt).content
            data['content'] = response
            SOURCES.append(data)
            with open(f"test.json", "w", encoding="utf-8") as f:
                json.dump(SOURCES, f, ensure_ascii=False, indent=4)

/workspace/kobc-agent-service/module/report_agent/retrieval_module/data/raw/drybulk_cargo/shippingnewsnet.json


  0%|          | 0/1780 [00:00<?, ?it/s]

  0%|          | 8/1780 [00:18<1:09:34,  2.36s/it]


KeyboardInterrupt: 

In [14]:
SOURCES[0]['content'].content

'The Capesize market experienced a turbulent week, marked by a mix of sporadic positivity and mounting downward pressure. The BCI 5TC started the week at $21,673, gained ground midweek, peaking at $23,049 on Tuesday, but plunged to $17,686 by Friday, losing $3,987 over the week. \n\nIn the Pacific, initial optimism driven by miner activity was dampened by persistent weather disruptions'